### Урок 3. Part-of-Speech разметка, NER , извлечение отношений#

### -- Автор: Шенк Евгений Станиславович

##### Тема “Улучшение качества  POS-taggera”
на вебинаре был рассмотрен пример тегирования с использованием корпуса на Русском языке, вам необходимо улучшить модель что бы качество классификации было выше чем с лог регрессией 

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

import pyconll
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
import urllib.request

# url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu'
# urllib.request.urlretrieve(url, './datasets/ru_syntagrus-ud-train.conllu')

# url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu'
# urllib.request.urlretrieve(url, './datasets/ru_syntagrus-ud-dev.conllu')

#### Train-Test

In [3]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [4]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [5]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

#### LabelEncoder

In [6]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

##### Vectorizer

In [7]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

In [8]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

#### Logistic Regression

In [9]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

LogisticRegression(max_iter=10, random_state=0)

In [10]:
pred_lr = lr.predict(X_test)

In [11]:
accuracy_score(test_enc_labels, pred_lr)

0.5258484143834462

#### Bigram tagger

In [12]:
unigram_tagger = UnigramTagger(fdata_train)
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.evaluate(fdata_test))

0.8829996966939642

#### XGBoost

In [13]:
xgb_model = xgb.XGBClassifier(learning_rate=0.05, 
                              max_depth=15,
                              n_estimators=200, 
                              nthread=-1,
                              objective='binary:logistic',
                              reg_alpha=0,
                              reg_lambda=0,
                              random_state=0, 
                              subsample=1)

In [14]:
xgb_model.fit(X_train, train_enc_labels)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=0, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [15]:
pred_xgb = xgb_model.predict(X_test)

In [16]:
accuracy_score(test_enc_labels, pred_xgb)

0.9423718531998787

##### Лучший результат у XGBoost (0.942)

### Тема “Создание признакового пространства”

#### Задание 1 и 2
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)  
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?  
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.  

### Spacy

In [17]:
import spacy
from spacy import displacy
import en_core_web_md

In [18]:
data = pd.read_pickle("../data/tweet_dataframe.pkl")
data.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


In [19]:
Train, Test = train_test_split(data, train_size=0.7, random_state=0)

In [20]:
train_text = "".join(str(x) for x in Train['clean_tweet'])
test_text = "".join(str(x) for x in Test['clean_tweet'])

In [21]:
text = "".join(str(x) for x in data['clean_tweet'])

In [49]:
nlp = en_core_web_md.load()
nlp.max_length = 4000000
#doc = nlp("I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm.")
doc = nlp(text)

nlp_train = en_core_web_md.load()
nlp_train.max_length = 4000000
doc_train = nlp_train(train_text)

nlp_test = en_core_web_md.load()
nlp_test.max_length = 4000000
doc_test = nlp_test(test_text)

In [55]:
# Отображение (закоменченно т.к. сильно увеличивает размер нотбука)
# displacy.render(doc, style="ent")

In [52]:
ents_df = pd.DataFrame(columns=['ents', 'label'])
for i, token in enumerate(doc.ents):
    ents_df.loc[i] = [str(token), str(token.label_)]
    
ents_df_train = pd.DataFrame(columns=['ents', 'label'])
for i, token in enumerate(doc_train.ents):
    ents_df_train.loc[i] = [str(token), str(token.label_)]
    
ents_df_test = pd.DataFrame(columns=['ents', 'label'])
for i, token in enumerate(doc_test.ents):
    ents_df_test.loc[i] = [str(token), str(token.label_)]

In [44]:
ents_df['ents'].groupby(by=ents_df['ents']).count().sort_values(ascending=False).head(20)

ents
today       1599
friday       619
first        612
tomorrow     576
one          533
sunday       489
tonight      451
orlando      428
summer       419
saturday     319
monday       241
morning      230
america      198
two          195
weekend      181
june         180
us           179
london       176
thursday     167
days         142
Name: ents, dtype: int64

In [45]:
# Total
ents_org_person = ents_df[ents_df['label'].isin(['ORG', "PERSON"])]
ents_org_person['ents'].groupby(by=ents_df['ents']).count().sort_values(ascending=False).head(20)

ents
obama                  108
gop                     85
hu                      68
hillary                 68
nba                     66
orlando                 56
bing bong bing bong     52
disney                  49
bing bong               45
isis                    43
stas                    42
blm                     39
lebron                  38
netflix                 38
christina grimmie       35
donald                  32
jesus                   29
nyc                     28
amazon                  28
cavs                    27
Name: ents, dtype: int64

In [53]:
# Train
ents_org_person = ents_df_train[ents_df_train['label'].isin(['ORG', "PERSON"])]
ents_org_person['ents'].groupby(by=ents_org_person['ents']).count().sort_values(ascending=False).head(20)

ents
obama                  86
orlando                55
hu                     54
gop                    51
hillary                46
bing bong bing bong    39
bing bong              35
stas                   35
netflix                34
isis                   31
christina grimmie      29
disney                 28
nba                    27
lebron                 27
nyc                    25
jesus                  24
blm                    23
sun                    23
islam                  21
god                    20
Name: ents, dtype: int64

In [54]:
# Test
ents_org_person = ents_df_test[ents_df_test['label'].isin(['ORG', "PERSON"])]
ents_org_person['ents'].groupby(by=ents_org_person['ents']).count().sort_values(ascending=False).head(20)

ents
obama                  30
nba                    29
gop                    24
hillary                24
orlando                19
disney                 17
bing bong bing bong    16
hu                     15
bing bong              14
cnn                    13
donald                 12
cavs                   12
blm                    12
nyc                    10
isis                   10
aap                     9
eu                      9
jo cox                  8
fbi                     8
islam                   8
Name: ents, dtype: int64

#### В топ попадают не только персоны и организации но и "bing bong"

### NLTK

In [34]:
set_of_ents = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text.upper()))) if hasattr(chunk, 'label') }
ents_df_nltk = pd.DataFrame(set_of_ents, columns=['ents', 'label'])

set_of_ents = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(train_text.upper()))) if hasattr(chunk, 'label') }
ents_df_nltk_train = pd.DataFrame(set_of_ents, columns=['ents', 'label'])

set_of_ents = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_text.upper()))) if hasattr(chunk, 'label') }
ents_df_nltk_test = pd.DataFrame(set_of_ents, columns=['ents', 'label'])

In [30]:
ents_df_nltk

,ents,label
0,INEEDIT,ORGANIZATION
1,REN,ORGANIZATION
2,PETAL,ORGANIZATION
3,BLACKMAGIC,ORGANIZATION
4,WISHES,ORGANIZATION
...,...,...
6215,DNC,ORGANIZATION
6216,MOVEMENT,ORGANIZATION
6217,HIRE,ORGANIZATION
6218,FURRBABY,ORGANIZATION


In [31]:
ents_df_nltk['ents'].groupby(by=ents_df_nltk['ents']).count().sort_values(ascending=False).head(20)

ents
HENDERSON      2
CLIENTS        2
IRELAND        2
WHITE          2
CHINA          2
US             2
JOHNSON        2
TODDLER        2
ISRAEL         2
FARRAH         1
FANS OF        1
FAN OF         1
FATAL          1
FAT            1
FANEPICNESS    1
FASHION        1
FANG           1
FASCIST        1
FANS           1
FANTASTIC      1
Name: ents, dtype: int64

In [32]:
# Total
ents_nltk_org_person = ents_df_nltk[ents_df_nltk['label'].isin(['ORGANIZATION', "PERSON"])]
ents_nltk_org_person['ents'].groupby(by=ents_nltk_org_person['ents']).count().sort_values(ascending=False).head(20)

ents
HENDERSON        2
TODDLER          2
JOHNSON          2
CLIENTS          2
ZUMA             1
FARHAD           1
FANEPICNESS      1
FANG             1
FANS             1
FANS OF          1
FANTASTIC        1
FAR              1
FAREWELL         1
FARIDOON         1
FAMOUS FOR       1
FARMERS          1
FARMERSMARKET    1
FARMHOUSE        1
FARRAH           1
FASCISM          1
Name: ents, dtype: int64

In [35]:
# Train
ents_org_person = ents_df_nltk_train[ents_df_nltk_train['label'].isin(['ORG', "PERSON"])]
ents_org_person['ents'].groupby(by=ents_org_person['ents']).count().sort_values(ascending=False).head(20)

ents
WORSE TRAGEDIES        1
CLINTON NOMINEE        1
CLINTON MAKES ME       1
CLINTON JIMMY          1
CLINTON IT             1
CLINTON INSPIRED       1
CLINTON HATES          1
CLINTON FOR            1
CLINTON DEFENDED       1
CLINTON BLACK          1
CLINTON ARE            1
CLINTON AM             1
CLINTON                1
CLINE AK               1
CLIFFHANGER FCKU       1
CLIFF HOW              1
CLICK TO WATCH LOTS    1
CLINTON MY             1
CLINTON SAY            1
WOODROWWILSON HELD     1
Name: ents, dtype: int64

In [36]:
# Test
ents_org_person = ents_df_nltk_test[ents_df_nltk_test['label'].isin(['ORG', "PERSON"])]
ents_org_person['ents'].groupby(by=ents_org_person['ents']).count().sort_values(ascending=False).head(20)

ents
WILLIAM AND          1
TODDLER              1
IMWITHHER BE         1
HENDERSON            1
HAPPY MOOD           1
GARRETT HAS          1
ELECT                1
DONALD TRUMP         1
DAVID PROPER         1
CLINTON RALLY        1
CLINTON MADE         1
CLINTON HE           1
CLINTON              1
CLIFTON VIDEO MEN    1
CLIENTS              1
CLICK TO WATCH AM    1
CLICK TO WATCH       1
CLICK TO DOWNLOAD    1
CLICK TO             1
CLICK SHOULD         1
Name: ents, dtype: int64

### Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.

In [37]:
#Распознано Spacy:  # 33791
ents_df.shape[0]

33791

In [38]:
ents_df['ents'].groupby(by=ents_df['ents']).count().sort_values(ascending=False).head(20)

ents
today       1599
friday       619
first        612
tomorrow     576
one          533
sunday       489
tonight      451
orlando      428
summer       419
saturday     319
monday       241
morning      230
america      198
two          195
weekend      181
june         180
us           179
london       176
thursday     167
days         142
Name: ents, dtype: int64

In [39]:
#Распознано NLTK:  # 13668
ents_df_nltk.shape[0]

13668

In [40]:
ents_df_nltk['ents'].groupby(by=ents_df_nltk['ents']).count().sort_values(ascending=False).head(20)

ents
AMERICANS    2
FATHER       2
TRUMP        2
PERSON       2
MADISON      2
IRAQ         2
IRELAND      2
OBAMA        2
CLINTON      2
BELGIAN      2
US           2
ALBUM        2
JOHN         2
CLIENTS      2
TURKEY       2
BERNIE       2
BARRY        2
INDIAN       2
ISRAEL       2
JOHNSON      2
Name: ents, dtype: int64

#### Качество Spacy оказалось лучше, количество распознаваний выше, релевантность выше, отработал быстрее. В NLTK пришлось сделать весь текст заглавными буквами, иначе результата не было вовсе.